# Курсовая работа 
## на тему
## "Машинное обучение в задачах распознавания голоса"



Определение пола человека (мужского или женского) на основе образца его голоса поначалу кажется простой задачей. Зачастую человеческое ухо легко уловит разницу между мужским и женским голосом уже с первых нескольких произнесенных слов.   
Модель построена с использованием 3168 записанных образцов мужских и женских голосов, речи и высказываний. Образцы обрабатываются с помощью акустического анализа, а затем применяются к алгоритму искусственного интеллекта/машинного обучения для изучения гендерных особенностей.

Для этой цели мы собираемся использовать разные модели: От классических моделей классификации до Keras.

In [ ]:
import numpy as np
import pandas as pd
from io import StringIO

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score, plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve, cross_val_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from scipy.interpolate import UnivariateSpline

import warnings
warnings.filterwarnings('ignore')

kfold = KFold(n_splits=5)

In [ ]:
# Random Forest feature importance Function
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols': df.columns, 'Importance':m.feature_importances_}
                       ).sort_values('Importance', ascending=False)

In [ ]:
# Random Forest feature importance 
def plot_fi(fi):
    return fi.plot('cols', 'Importance', 'barh', figsize=(12,8), legend=False)

## Исследовательский анализ данных

In [ ]:
path = "/kaggle/input/voicegender/voice.csv"
voiceDf = pd.read_csv(path)

voiceDf.head()

In [ ]:
voiceDf.describe()

In [ ]:
voiceDf.shape

### Описание набора данных
- Набор данных содержит 21 столбец (20 атрибутов и 1 целевой признак - пол)
- Размеры набора данных: 3168 x 21

- __Цель: Классифицировать пол на основе особенностей голоса.__  

### Признаки и таргет набора данных:
- **meanfreq**: средняя частота (в кГц)
- **sd**: стандартное отклонение частоты
- **median**: медианная частота (в кГц)
- **Q25**: первый квантиль (в кГц)
- **Q75**: третий квантиль (в кГц)
- **IQR**: интерквартильный размах (в кГц)
- **skew**: асимметрия
- **kurt**: эксцесс
- **sp.ent**: спектральная энтропия
- **sfm**: спектральная плоскость
- **centroid**: частотный центроид (см. specprop)
- **peakf**: пиковая частота (частота с наибольшей энергией)
- **meanfun**: среднее значение основной частоты, измеренной в акустическом сигнале
- **minfun**: минимальная основная частота, измеренная в акустическом сигнале
- **maxfun**: максимальная основная частота, измеренная в акустическом сигнале
- **meandom**: среднее значение доминирующей частоты, измеренной в акустическом сигнале
- **mindom**: минимальная доминирующая частота, измеренная в акустическом сигнале
- **maxdom**: максимальная доминирующая частота, измеренная в акустическом сигнале
- **dfrange**: диапазон доминирующей частоты, измеренной в акустическом сигнале
- **modindx**: индекс модуляции. Накопленная абсолютная разница между соседними измерениями основных частот, деленная на диапазон частот
- **label**: пол (мужчина или женщина)


### Подготовка данных

In [ ]:
voiceDf.dtypes.unique()

**Все признаки - численные, таргет - object.**

In [ ]:
voiceDf.isnull().sum()

**Во всех признаках отсутствуют пропуски.**

Далее преобразуем целевую переменную в численный тип.  
Воспользуемся label_encoder, ведь у нас только 2 пола:

In [ ]:
voiceDf["label"] = LabelEncoder().fit_transform(voiceDf["label"]) # 1 - male , 0 - female

### Визуализация данных

In [ ]:
sns.pairplot(voiceDf[['meanfreq', 'Q25', 'Q75', 'skew', 'centroid', 'label']], hue='label');

**Pairplot построен для демонстрации взаимосвязей между признаками в наборе данных. Мы видим, что асимметрия и квартили имеют слабую связь, в отличие от "meanfreq" и "центроида", между которыми наблюдается сильная связь.**

In [ ]:
voiceDf.hist(figsize=(21, 10))

plt.tight_layout()
plt.show()

In [ ]:
voiceDf.boxplot(figsize=(21, 10))

plt.show()

Выше представлены столбчатые диаграммы и boxplot каждого атрибута, которые позволяют увидеть наличие выбросов в данных. Изучив их, мы можем заметить, что __данные содержат не много выбросов__. Наибольшее количество выбросов наблюдается в признаках `skew` (асимметрия) и `kurt` (эксцесс).   
Мы видим, что __данные не смещены__, так как они __равномерно распределены между классами__ (50% мужчин, 50% женщин) - __дисбаланс классов отсутствует__.

In [ ]:
f, ax = plt.subplots(figsize=(16, 10))
corr = voiceDf.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), 
            cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax, annot=True)

plt.title('Матрица корреляций', fontsize=20)
plt.tight_layout()
plt.show()

**Эта тепловая карта показывает корреляции между признаками.   
Заметим положительную корреляцию между label и IQR, label и sp.ent, а также сильную отрицательную корреляцию между label и meanfun.**

## Предобработка данных

**Сначала мы разделяем набор данных на объекты, классы**

In [ ]:
gender_label = voiceDf.iloc[:, -1].to_numpy()
voiceDf1 = voiceDf.iloc[:, :-1]

**Затем делим на тренировочную и тестовые выборки**

In [ ]:
voiceX_train, voiceX_test, voiceY_train, voiceY_test = train_test_split(voiceDf1, gender_label, test_size=0.3, random_state=42)

**Стандартизация признаков**
- Обучаем scaler на тренировочных признаках
- Трансформируем тестовые. 


In [ ]:
scaler = StandardScaler()
voiceX_train = scaler.fit_transform(voiceX_train)
voiceX_test = scaler.transform(voiceX_test)

## Модели машинного обучения

В этом разделе мы обучим несколько моделей классификации, оценим их, составим итоговую таблицу метрик для общих выводов.

### Логистическая регрессия 

Применим для нахождения лучших параметров поиск по сетке

In [ ]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'penalty': ['l1', 'l2'],
              'max_iter': list(range(100, 800, 100)),
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
%time grid_search.fit(voiceX_train, voiceY_train)

print("Лучшие параметры: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

Поиск по сетке показал, что в целом значения по умолчанию в LogisticRegression подходят для лучшего результата, изменив только гиперпараметр ```solver``` на 'saga', что объясняется размерностью датасета.

##### Обучение и предсказание с использованием лучшей модели логистической регрессии

In [ ]:
best_logreg = grid_search.best_estimator_
voice_pred_best = best_logreg.predict(voiceX_test)

# Оценка метрик для найденной модели
voice_score_logreg = accuracy_score(voice_pred_best, voiceY_test)
accuracy_results_logreg = cross_val_score(best_logreg, voiceX_train, voiceY_train, cv=kfold).mean()
prf_logreg = precision_recall_fscore_support(voiceY_test, voice_pred_best, average='macro')
Precision_logreg = prf_logreg[0]
Recall_logreg = prf_logreg[1]
f1_logreg = prf_logreg[2]

##### Визуализация матрицы ошибок для лучшей модели логистической регрессии

In [ ]:
plot_confusion_matrix(best_logreg, voiceX_test, voiceY_test, cmap='PuBuGn')
plt.title('Confusion matrix of logreg')
plt.show()

### К-Ближайших соседей (KNN)

In [ ]:
# сначала найдем оптимальное n_neighbors
knn_valid_score_list = []
n_neighbors_num = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

for i in range(1,15):
    test_knn = KNeighborsClassifier(n_neighbors=i)
    test_knn.fit(voiceX_train, voiceY_train)
    knn_valid_score_list.append(test_knn.score(voiceX_test, voiceY_test))
    
plt.plot(n_neighbors_num,  knn_valid_score_list, "b")
plt.plot(n_neighbors_num,  knn_valid_score_list, "bo")
plt.xlabel("Количество соседей")
plt.ylabel("Score %")
plt.grid(True)
plt.show()

**Лучший результат при n_neighbors = 4, используем такое значение параметра.** 

In [ ]:
# Учим лучшую модель KNN
%time knn = KNeighborsClassifier(n_neighbors=4).fit(voiceX_train, voiceY_train)

voice_pred_knn=knn.predict(voiceX_test)
prf_knn=precision_recall_fscore_support(voiceY_test, voice_pred_knn, average='macro')

# Смотрим метрики 
voice_score_knn = accuracy_score(voice_pred_knn, voiceY_test)
accuracy_results_knn = cross_val_score(knn, voiceX_train, voiceY_train, cv=kfold).mean()
Precision_knn = prf_knn[0]
Recall_knn = prf_knn[1]
f1_knn = prf_knn[2]

После вычисления метрик строим confusion_matrix

In [ ]:
plot_confusion_matrix(knn, voiceX_test, voiceY_test, cmap='PuBuGn')
plt.title('Confusion matrix of the KNN')
plt.show()

### Метод Опорных Векторов (SVM)

In [ ]:
%time svm = SVC().fit(voiceX_train, voiceY_train)

voice_pred_svm=svm.predict(voiceX_test)
prf_svm=precision_recall_fscore_support(voiceY_test, voice_pred_svm, average='macro')

voice_score_svm = accuracy_score(voice_pred_svm, voiceY_test)
accuracy_results_svm = cross_val_score(svm, voiceX_train, voiceY_train, cv=kfold).mean()
Precision_svm = prf_svm[0]
Recall_svm = prf_svm[1]
f1_svm = prf_svm[2]

In [ ]:
plot_confusion_matrix(svm, voiceX_test, voiceY_test, cmap='PuBuGn')
plt.title('Confusion matrix of the SVM')
plt.show()

### Случайный лес (Random Forest)

In [ ]:
%time rf = RandomForestClassifier(n_estimators=100).fit(voiceX_train, voiceY_train)

voice_pred_rf = rf.predict(voiceX_test)
prf_rf=precision_recall_fscore_support(voiceY_test, voice_pred_rf, average='macro')

voice_score_rf = accuracy_score(voice_pred_rf, voiceY_test)
accuracy_results_rf = cross_val_score(rf, voiceX_train, voiceY_train, cv=kfold).mean()
Precision_rf = prf_rf[0]
Recall_rf = prf_rf[1]
f1_rf = prf_rf[2]

In [ ]:
plot_confusion_matrix(rf,voiceX_test, voiceY_test, cmap='PuBuGn')
plt.title('Confusion matrix of the Random Forest')
plt.show()

**Также мы можем определить важность каждого признака согласно Random Forest Classifier и отобразить это на линейном графике.**

In [ ]:
rf_importance = rf_feat_importance(rf, voiceDf1)
rf_importance.sort_values(by='Importance', ascending=False)

In [ ]:
plot_fi(rf_importance);

**Так, ```meanfun``` является наиболее важным признаком среди всех, затем следует ```IQR```. Это вполне логично.**

## Кривые обучения
Строим графики кривых обучения для каждой модели, чтобы дать дополнительную информацию о производительности моделей

In [ ]:
logreg_train_sizes, logreg_train_scores, logreg_valid_scores, *_ = learning_curve(best_logreg, voiceX_train, voiceY_train, n_jobs=-1,
                                                        random_state=42, cv=3)

knn_train_sizes, knn_train_scores, knn_valid_scores, *_ = learning_curve(knn, voiceX_train, voiceY_train, n_jobs=-1,
                                                        random_state=42, cv=3)

svm_train_sizes, svm_train_scores, svm_valid_scores, *_ = learning_curve(svm, voiceX_train, voiceY_train, n_jobs=-1,
                                                        random_state=42, cv=3)

rf_train_sizes, rf_train_scores, rf_valid_scores, *_ = learning_curve(rf, voiceX_train, voiceY_train, n_jobs=-1,
                                                        random_state=42, cv=3)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Learning Curve', fontsize=18)

# Графики обучения
sns.scatterplot(ax=axes[0], x= logreg_train_sizes, y= logreg_train_scores.mean(axis=1))
sns.lineplot(ax=axes[0], x= logreg_train_sizes, y= logreg_train_scores.mean(axis=1))
sns.scatterplot(ax=axes[0], x= rf_train_sizes, y= rf_train_scores.mean(axis=1))
sns.lineplot(ax=axes[0], x= rf_train_sizes, y= rf_train_scores.mean(axis=1))
sns.scatterplot(ax=axes[0], x= svm_train_sizes, y= svm_train_scores.mean(axis=1))
sns.lineplot(ax=axes[0], x= svm_train_sizes, y= svm_train_scores.mean(axis=1))
sns.scatterplot(ax=axes[0], x= knn_train_sizes, y= knn_train_scores.mean(axis=1))
sns.lineplot(ax=axes[0], x= knn_train_sizes, y= knn_train_scores.mean(axis=1))
axes[0].set_title('Train')
axes[0].set_xlabel('Data size')
axes[0].set_ylabel('Score')
axes[0].legend(['LogReg', 'RF', 'SVM', 'KNN'])

# Графики тестовых 
sns.scatterplot(ax=axes[1], x= logreg_train_sizes, y= logreg_valid_scores.mean(axis=1))
sns.lineplot(ax=axes[1], x= logreg_train_sizes, y= logreg_valid_scores.mean(axis=1))
sns.scatterplot(ax=axes[1], x= rf_train_sizes, y= rf_valid_scores.mean(axis=1))
sns.lineplot(ax=axes[1], x= rf_train_sizes, y= rf_valid_scores.mean(axis=1))
sns.scatterplot(ax=axes[1], x= svm_train_sizes, y= svm_valid_scores.mean(axis=1))
sns.lineplot(ax=axes[1], x= svm_train_sizes, y= svm_valid_scores.mean(axis=1))
sns.scatterplot(ax=axes[1], x= knn_train_sizes, y= knn_valid_scores.mean(axis=1))
sns.lineplot(ax=axes[1], x= knn_train_sizes, y= knn_valid_scores.mean(axis=1))
axes[1].set_title('Valid')
axes[1].set_xlabel('Data size')
axes[1].set_ylabel('Score')
axes[1].legend(['LogReg', 'RF', 'SVM', 'KNN'])

plt.show()

### Таблица оценки производительности моделей

Ниже представленная таблица сравнивает примененные модели с использованием различных метрик оценки: model.score, кросс-валидация, точность, полнота, F1-оценка,  `model.score`, `Cross Validation`,  `Precision`, `Recall`, `F1 Score` чтобы увидеть, какая модель является оптимальной для этого набора данных.

In [ ]:
model_performance_table = pd.DataFrame({
    'Model': ['SVM', 'KNN', 'RF', 'LogReg'],
    'Model Score': [voice_score_svm, voice_score_knn, voice_score_rf, voice_score_logreg],
    'Cross Validation': [accuracy_results_svm, accuracy_results_knn, accuracy_results_rf, accuracy_results_logreg],
    'Valid Precision': [Precision_svm, Precision_knn, Precision_rf, Precision_logreg],
    'Valid Recall': [Recall_svm, Recall_knn, Recall_rf, Recall_logreg],
    'Valid F1 Score': [f1_svm, f1_knn, f1_rf, f1_logreg]
})

model_performance_table.sort_values(by="Model Score", ascending=False)

---
SVM оказалась лидером по всем метрикам, включая точность модели, точность кросс-валидации, точность валидации, полноту валидации и F1-оценку валидации. Это говорит о высокой надежности и эффективности этой модели.

KNN показала немного меньшие результаты по сравнению с SVM, но все равно достаточно высокие, чтобы считать эту модель эффективной. Она может быть хорошим выбором в случаях, когда SVM не подходит по каким-либо причинам.

В общем, все модели показали хорошие результаты, что говорит об их эффективности в решении задачи распознавании голоса, а классификации пола по голосу.

---

## TensorFlow

### Создание Keras Sequential модели

In [ ]:
def create_model():
   
    model = Sequential()
    model.add(Dense(256, input_shape=x_train.shape))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3))
    # Один выходной нейрон с сигмоидной функцией активации
    model.add(Dense(1, activation="sigmoid"))
    # используем функцию потерь binary_crossentropy
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
    
    model.summary()
    return model

1. Создается последовательная модель `Sequential`.
2. Добавляется полносвязный слой с 256 нейронами. Входная форма для этого слоя - размерность обучающих данных.
3. Добавляется слой Dropout с параметром 0.3. Так, во время обучения 30% нейронов в предыдущем слое будут случайно отключаться на каждом шаге обучения, чтобы предотвратить переобучение.
4. Добавляются еще три пары слоев Dense и Dropout для дополнительного обучения и регуляризации модели.
5. Добавляется последний слой Dense с одним нейроном и функцией активации `sigmoid`. Это делает модель подходящей для бинарной классификации (в данном случае, определение пола).
6. Модель компилируется с функцией потерь `binary_crossentropy` (для бинарной классификации).

In [ ]:
model = create_model()

##### Предупреждаем переобучение

Останавливаем обучение модели при достижении 95% точности на обучающей выборке.

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    # Вызываем в конце каждой эпохи
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95):
            print("nДостигнута точность 95%, - дальнейшее обучение отменяется!")
            self.model.stop_training = True   # Останавливаем обучение


In [ ]:
callbacks = myCallback()

#### Обучаем Sequantial модель

In [ ]:
batch_size = 64
epochs = 100
# train the model using the training set and validating using validation set
model.fit(x_train, y_train, 
          epochs=epochs,
          batch_size=batch_size,
          callbacks=[callbacks])